In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import t
import matplotlib.pyplot as plt
from arch import arch_model

In [2]:
np.random.seed(11)
ticker = "TRP.RO"
start_date = "2021-01-01"
confidence = 0.99
try:
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date)
except Exception as e:
    print("Error downloading", e)
    df = None
if df is None or df.empty:
    raise ValueError(f"No data returned for {ticker}")
if "Adj Close" in df.columns:
    df = df[["Adj Close"]].rename(columns={"Adj Close": "stock"})
else:
    df = df[["Close"]].rename(columns={"Close": "stock"})
prices_log = np.log(1 + df["stock"].pct_change())
prices_log.replace([np.inf, -np.inf], np.nan, inplace=True)
prices_log.dropna(inplace=True)
current_price = float(df["stock"].iloc[-1])
r = prices_log * 100
print(df)
print(r)

                              stock
Date                               
2021-01-04 00:00:00+02:00  0.235726
2021-01-05 00:00:00+02:00  0.234644
2021-01-06 00:00:00+02:00  0.237348
2021-01-07 00:00:00+02:00  0.238970
2021-01-08 00:00:00+02:00  0.238429
...                             ...
2026-02-09 00:00:00+02:00  0.499000
2026-02-10 00:00:00+02:00  0.504000
2026-02-11 00:00:00+02:00  0.521000
2026-02-12 00:00:00+02:00  0.521000
2026-02-13 00:00:00+02:00  0.512000

[1275 rows x 1 columns]
Date
2021-01-05 00:00:00+02:00   -0.460009
2021-01-06 00:00:00+02:00    1.145705
2021-01-07 00:00:00+02:00    0.681039
2021-01-08 00:00:00+02:00   -0.226615
2021-01-11 00:00:00+02:00   -0.454424
                               ...   
2026-02-09 00:00:00+02:00    0.401608
2026-02-10 00:00:00+02:00    0.997016
2026-02-11 00:00:00+02:00    3.317381
2026-02-12 00:00:00+02:00    0.000000
2026-02-13 00:00:00+02:00   -1.742542
Name: stock, Length: 1274, dtype: float64


In [3]:
model = arch_model(r, vol='Garch', p=1, q=1, dist='t', mean='Constant')
res = model.fit(disp='off')
forecasts = res.forecast(horizon=1)
next_mean = forecasts.mean.iloc[-1].values[0] / 100
next_var = forecasts.variance.iloc[-1].values[0] / (100**2)
next_sigma = np.sqrt(next_var)
nu = res.params['nu'] # Degrees of freedom
alpha = 1 - confidence
t_cutoff = t.ppf(alpha, df=nu)
var_log_return = -(next_mean + next_sigma * t_cutoff)
garch_var_price = current_price * (1 - np.exp(-var_log_return))
garch_var_pct = (1 - np.exp(-var_log_return)) * 100

ES = -(mean + sigma * nu + (t_pdf / t_cdf) * sigma * (nu + t_cutoff^2)/(nu-1))

In [4]:
t_pdf = t.pdf(t_cutoff, df=nu)
t_cdf = t.cdf(t_cutoff, df=nu)
es_log_return = -(next_mean + next_sigma * (nu + t_cutoff**2)/(nu-1) * t_pdf/t_cdf)
garch_es_price = current_price * (1 - np.exp(-es_log_return))
garch_es_pct = (1 - np.exp(-es_log_return)) * 100

Print results

In [5]:
print(f"\nTicker: {ticker}")
print(f"Last price: {current_price:.4f}")
print(f"GARCH(1,1) Student-t Volatility (Next Day): {next_sigma*100:.4f}%")
print(f"Estimated Degrees of Freedom (nu): {nu:.4f}")
print(f"GARCH Student-t VaR ({confidence*100:.0f}%): {garch_var_price:.4f} RON ({garch_var_pct:.4f}%)")
print(f"On {confidence*100:.0f}% of days, you should expect not to lose more than {garch_var_pct:.4f}% in a day")
print(f"GARCH Student-t Expected Shortfall ({confidence*100:.0f}%): {garch_es_price:.4f} RON ({garch_es_pct:.4f}%)")


Ticker: TRP.RO
Last price: 0.5120
GARCH(1,1) Student-t Volatility (Next Day): 2.1735%
Estimated Degrees of Freedom (nu): 3.0369
GARCH Student-t VaR (99%): 0.0481 RON (9.3960%)
On 99% of days, you should expect not to lose more than 9.3960% in a day
GARCH Student-t Expected Shortfall (99%): -0.0823 RON (-16.0669%)
